In [1]:
from speech_utils import SpeechConverter, sr
from TTS_DataLoader import text_to_seq_char_level, symbols_len
from Models.TtsSimple import TTS_Simple
import torch
from IPython.display import Audio, display
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Inference running on device={device}')


/home/sid/Desktop/wpi/kdd/Text-2-Audio/kdd4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Inference running on device=cuda


In [13]:
input_raw_text = " "
# TODO: need to write a custom text normalization function
# normalization minimum functionality => lower case, number to text, filter unknown vocab in TTS_DataLoader vocab dict
input_indexed = text_to_seq_char_level(input_raw_text) # throw error in text_to_seq for invalid input
input_indexed = input_indexed.unsqueeze(0).to(device)
print(input_indexed)
print(input_indexed.shape)

tensor([[1, 0]], device='cuda:0', dtype=torch.int32)
torch.Size([1, 2])


In [14]:
# import based on h_params from main
mel_bins = 96
embedding_dim = 256
enc_out_size = 512
model_path = "./checkpoints/TtsSimple_Train.pt"
model = TTS_Simple(device, symbols_len, embedding_dim, enc_out_size, mel_bins)
model.load_state_dict(torch.load(model_path, weights_only=True))
model.to(device)

TTS_Simple(
  (enc_embedding): Embedding(43, 256)
  (enc_conv1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (enc_batch_norm_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_dropout_1): Dropout(p=0.0, inplace=False)
  (enc_conv2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (enc_batch_norm_2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_dropout_2): Dropout(p=0.0, inplace=False)
  (enc_lstm): LSTM(256, 512, batch_first=True, bidirectional=True)
  (dec_lstm): LSTM(96, 1024, batch_first=True)
  (dec_lin_proj): Linear(in_features=1024, out_features=96, bias=True)
  (dec_eos_gate): Linear(in_features=1024, out_features=1, bias=True)
)

In [15]:
out = model.inference(input_indexed)
out = out.to('cpu')
print(out.shape)

torch.Size([1, 800, 96])


In [16]:
SC = SpeechConverter(mel_bins)
audio_waveform = SC.inverse_mel_spec_to_wav(out.permute(0,2,1))


In [17]:
aud = Audio(data=audio_waveform, rate=sr)
display(aud)